In [ ]:
import warnings

warnings.filterwarnings("ignore")

import sqlite3  # parse trajectory db
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure


import pedpy


def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed, midpoint):
    colorscale = px.colors.diverging.RdBu_r[::-1]

    # Normalize speed based on the midpoint
    if speed >= midpoint:
        normalized_speed = 0.5 + 0.5 * (speed - midpoint) / (
            max_speed - midpoint
        )
    else:
        normalized_speed = 0.5 * (speed - min_speed) / (midpoint - min_speed)

    # Clip to ensure the value is between 0 and 1
    normalized_speed = np.clip(normalized_speed, 0, 1)

    # Find the corresponding color in the colorscale
    color_idx = int(normalized_speed * (len(colorscale) - 1))
    return colorscale[color_idx]


def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces


def get_shapes_for_frame(frame_data, min_speed, max_speed, midpoint):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == -1000:  # Check for dummy speed
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                hover_trace,
            )
        color = speed_to_color(row["speed"], min_speed, max_speed, midpoint)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace,
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    return shapes, hover_traces


def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_hover_trace,
    geometry_traces,
    hover_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    minx, miny, maxx, maxy = area_bounds
    fig = go.Figure(
        data=geometry_traces + hover_traces + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes,
            title=f"<b>Number of Agents: {initial_agent_count}</b>",
            title_x=0.5,
        ),
    )
    fig.update_layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [
                            None,
                            {
                                "frame": {"duration": 100, "redraw": True},
                                "fromcurrent": True,
                            },
                        ],
                        "label": "Play",
                        "method": "animate",
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=[
            {
                "active": 0,
                "yanchor": "top",
                "xanchor": "left",
                "currentvalue": {
                    "font": {"size": 20},
                    "prefix": "Frame:",
                    "visible": True,
                    "xanchor": "right",
                },
                "transition": {"duration": 100, "easing": "cubic-in-out"},
                "pad": {"b": 10, "t": 50},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": steps,
            }
        ],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )
    return fig


def animate(
    data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=5
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    midpoint = np.mean(data_df["speed"])
    max_agents = data_df.groupby("frame").size().max()
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": -1000}
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hovers = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed, midpoint
    )
    for frame_num in selected_frames[1:]:
        frame_data = data_df[data_df["frame"] == frame_num]
        agent_count = len(frame_data)
        while len(frame_data) < max_agents:
            dummy_df = pd.DataFrame([dummy_agent_data])
            frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)

        shapes, hover_traces = get_shapes_for_frame(
            frame_data, min_speed, max_speed, midpoint
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes,
                title=f"<b>Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)
        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_hovers,
        geometry_traces,
        hover_traces,
        frames,
        steps,
        area.bounds,
        width=800,
        height=800,
    )

# Getting started

Welcome to the getting started guide of *JuPedSim*, we want to guide you through the first steps for setting up a simulation.

First things first, to use *JuPedSim* install it via:

```bash
pip install jupedsim
```

Now, you are ready to set up your first simulation with *JuPedSim*.

If you want to follow this Jupyter Notebook on your own machine, you can download it 
{download}`here <getting_started.ipynb>`.

## Let's simulate an experiment

This is a bottleneck experiment conducted at the University of Wuppertal in 2018.
You can see the basic setup of the experiment in the picture below:

```{eval-rst}
.. figure:: demo-data/bottleneck/040_c_56_h-.png
    :width: 400px
    :align: center
```

The data for this experiment is available {download}`here <demo-data/bottleneck/040_c_56_h-.txt>`, which belongs to this [experimental series](https://doi.org/10.34735/ped.2018.1) and is part of the publication ["Crowds in front of bottlenecks at entrances from the perspective of physics and social psychology"](https://doi.org/10.1098/rsif.2019.0871).

In this guide, we want to simulate the same bottleneck scenario with *JuPedSim*.

If you use *JuPedSim* in your work, please cite it using the following information from Zenodo:

[![DOI](https://zenodo.org/badge/DOI/10.5281/1293771.svg)](https://doi.org/10.5281/zenodo.1293771)



## Setting up the simulation geometry

The first thing to consider when setting up a simulation is the area where the agents can move.
In this scenario, we have a rectangular room with two obstacles that form a bottleneck, as displayed above.
These obstacles are excluded areas from the simulation, which means that agents can not move inside these obstacles.

In [ ]:
import shapely

geometry = shapely.Polygon(
    # complete area
    [
        (3.5, -2),
        (3.5, 8),
        (-3.5, 8),
        (-3.5, -2),
    ],
    holes=[
        # left barrier
        [
            (-0.7, -1.1),
            (-0.25, -1.1),
            (-0.25, -0.15),
            (-0.4, 0.0),
            (-2.8, 0.0),
            (-2.8, 6.7),
            (-3.05, 6.7),
            (-3.05, -0.3),
            (-0.7, -0.3),
            (-0.7, -1.0),
        ],
        # right barrier
        [
            (0.25, -1.1),
            (0.7, -1.1),
            (0.7, -0.3),
            (3.05, -0.3),
            (3.05, 6.7),
            (2.8, 6.7),
            (2.8, 0.0),
            (0.4, 0.0),
            (0.25, -0.15),
            (0.25, -1.1),
        ],
    ],
)

This leads to the following setup:

In [ ]:
import matplotlib.pyplot as plt
from pedpy import plot_walkable_area, WalkableArea

plot_walkable_area(walkable_area=WalkableArea(geometry)).set_aspect("equal")
plt.show()

In this geometric setup, we now need to define, where the agents can exit the simulation, e.g., where their target is located.

For this scenario, the exit is at the end of the bottleneck.
If we would put it at the beginning of the bottleneck, the agents would disappear, hence, they wouldn't be an obstacle for following agents.

An exit can be defined as:

In [ ]:
exit_polygon = shapely.Polygon(
    [(-0.2, -1.9), (0.2, -1.9), (0.2, -1.7), (-0.2, -1.7)]
)

The complete setup would then look like, where the exit is shown in red:

In [ ]:
import matplotlib.pyplot as plt
from pedpy import plot_measurement_setup, WalkableArea, MeasurementArea

# We will use PedPy's plotting functionality, but it has no concept of exits
# hence we will show the exit as measurement area
plot_measurement_setup(
    walkable_area=WalkableArea(geometry),
    measurement_areas=[MeasurementArea(exit_polygon)],
    ma_color="r",
    ma_line_color="r",
).set_aspect("equal")
plt.show()

## Distribute the agents in the geometry


The next step is to provide the initial positions of the agents in the simulation. 
*JuPedSim* provides different helper functions to automatically distribute the agents:

- distribute a given number of agents inside a polygon
- distribute agents by a given density inside a polygon

However, as we want to compare our simulation to the real-life experimental data, the agents in the simulation will start at the same positions as the pedestrians in the experiment. 
So we load the trajectory from the experiment and extract the locations at frame 0, which is the start of the experiment.

In [ ]:
import pandas

experiment_data = pandas.read_csv(
    "demo-data/bottleneck/040_c_56_h-.txt",
    comment="#",
    sep="\t",
    header=None,
    names=["id", "frame", "x", "y", "z"],
)
start_positions = experiment_data[experiment_data.frame == 0][
    ["x", "y"]
].values

## Collision-free speed model

At first, we want to run the simulation with the Collision-free speed model.
In ["Collision-free speed model for pedestrian dynamics"](https://arxiv.org/abs/1512.05597) you can find the complete description of the model.
To use it in *JuPedSim* we need to state, that we will use this model in the creation of the simulation object.

We also need to provide the previously defined geometry and the output file, where the resulting trajectories will be stored. 
Here, we will use the built-in `SqliteTrajectoryWriter`, which will write the results as a `sqlite`-database.

In [ ]:
import jupedsim as jps
import pathlib

trajectory_file = "bottleneck_cfsm.sqlite"
simulation_cfsm = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(),
    geometry=geometry,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

After we defined the simulation we need to add a goal, which the agents should target. 
As we already defined the exit polygon above, we can add it to the simulation as a stage.
After adding the exit to the simulation, we need to create a journey containing that exit.
Journeys are used to manage more complex routing situation, but this will be explained in more detail in some **TODO other guide**.

In [ ]:
exit_id = simulation_cfsm.add_exit_stage(exit_polygon)
journey = jps.JourneyDescription([exit_id])
journey_id = simulation_cfsm.add_journey(journey)

Now we have the setup for the simulation.
But there are no agents in simulation yet, so we need to add them.

Before adding a simulation we need to define the agent specific parameters. 
In this example, all the agents will get the same parameters.
We will now re-use the above extracted start positions of the agents.

In [ ]:
for position in start_positions:
    simulation_cfsm.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            journey_id=journey_id,
            stage_id=exit_id,
            position=position,
            radius=0.12,
        )
    )

After, we have added the agents. 
We've got everything needed to start a simulation.
Here, we will iterate, as long as there are agents inside the simulation, e.g., some agents have not yet reached the exit.

In [ ]:
while simulation_cfsm.agent_count() > 0:
    simulation_cfsm.iterate()

Now, every agent has left the simulation.
Let's have a look at the result:

In [ ]:
import pedpy

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
speed = pedpy.compute_individual_speed(traj_data=trajectory_data, frame_step=5)
speed = speed.merge(trajectory_data.data, on=["id", "frame"], how="left")

animate(trajectory_data, walkable_area)

We have set up a first simple simulation with *JuPedSim*.
The results are saved in trajectory files, which can be used for further analyzes. 


For more examples, how to set up simulations have a look at the following notebooks:

- [Bottleneck scenario](bottlenneck)
- [Corner scenario](corner)
- [Crossing scenario](crossing)
- [4-Way crossing scenario](crossing_4_directions)
- [Double bottleneck scenario](double-bottleneck)
- [Double bottleneck scenario (1m width)](double-bottleneck_1m-breit)

We would love to hear some [feedback](https://www.github.com/PedestrianDynamics/jupedsim/discussions) from you!

## Comparison

After simulating a bottleneck scenario with the initial positions as in the experimental setup, we now want to compare the simulation results with the experiment results.
The first step here, is to have a look at the trajectories of both:

In [ ]:
import pedpy
import pathlib
import matplotlib.pyplot as plt

experimental_trajectories = pedpy.load_trajectory(
    trajectory_file=pathlib.Path("demo-data/bottleneck/040_c_56_h-.txt")
)

cfsm_data, walkable_area = read_sqlite_file(trajectory_file)

fig = plt.figure()

ax1 = fig.add_subplot(121, aspect="equal")
ax1.set_title("experimental data")
pedpy.plot_trajectories(
    traj=experimental_trajectories,
    axes=ax1,
    walkable_area=pedpy.WalkableArea(geometry),
)

ax2 = fig.add_subplot(122, aspect="equal")
ax2.set_title("Collision free speed model")
pedpy.plot_trajectories(
    traj=cfsm_data, axes=ax2, walkable_area=pedpy.WalkableArea(geometry)
)
plt.show()

But we do not only want to have a qualitative comparision, but also a quantitive one.
We will use [PedPy](https://pedpy.readthedocs.io) for this analysis, it is a Python library designed for analysing pedestrian movement data.


So, we want to compare the flow of the data sets at the bottleneck.
First, we create a measurement line at the beginning of the bottleneck:

In [ ]:
import pedpy

measurement_line = pedpy.MeasurementLine([(0.35, 0), (-0.35, 0)])

In [ ]:
import pedpy

pedpy.plot_measurement_setup(
    walkable_area=walkable_area,
    measurement_lines=[measurement_line],
    ml_color="red",
    ml_width=2,
).set_aspect("equal")
plt.show()

For this line we can then compute the flow:

In [ ]:
import pedpy

nt_experiment, _ = pedpy.compute_n_t(
    traj_data=experimental_trajectories,
    measurement_line=measurement_line,
)

nt_cfsm, _ = pedpy.compute_n_t(
    traj_data=cfsm_data,
    measurement_line=measurement_line,
)

The results can be seen here:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("N-t")
ax.plot(
    nt_experiment["time"],
    nt_experiment["cumulative_pedestrians"],
    label="exeriment",
)
ax.plot(nt_cfsm["time"], nt_cfsm["cumulative_pedestrians"], label="cfsm")
ax.legend()
ax.set_xlabel("t / s")
ax.set_ylabel("# pedestrians")
plt.show()